# 5a- Run Metaclassifier
This code loads predictions from GLM, XGB, and NN on both sets B and C.  It uses the McNamara variable set.

It uses the predictions from set B as train data, and uses the predicitons from set C as test data.

It then uses this data in two Metaclassifiers- one using XGB and the other using LR.  Although LR is trained on this data, there is currently no output generated from that classifier.

Finally this model outputs the raw meta predictions on the test set (Set C) and saves that along with the other model's estimates on C as the file `trainCpreds_fig_mcnamara.Rdata` .

In [1]:
setwd("/data/Projects/ACC_NCDR/NCDR/BJMDATA/ACTION/NCHData/")
source(file = "/data/Projects/ACC_NCDR/NCDR/BJMDATA/ACTION/CODE/xgboostWrapper.R")
source(file = "/data/Projects/ACC_NCDR/NCDR/BJMDATA/ACTION/CODE/fMeasure.R")
source(file = "/data/Projects/ACC_NCDR/NCDR/BJMDATA/ACTION/CODE/calcDeciles.R")

#library(pROC)
library(PRROC)
library(caret)
library(xgboost)
library(glmnet)

Type 'citation("pROC")' for a citation.

Attaching package: ‘pROC’

The following objects are masked from ‘package:stats’:

    cov, smooth, var

Loading required package: foreach
Loading required package: iterators
Loading required package: parallel
Loading required package: lattice
Loading required package: ggplot2
Loading required package: Matrix
Loaded glmnet 2.0-18


Attaching package: ‘glmnet’

The following object is masked from ‘package:pROC’:

    auc



In [2]:
folds.all <- list(1, 2, 3, 4, 5)
modes <- list('mcnamara', 'expanded')

In [4]:
for (fold in folds.all){
    for (mode in modes){
        data_dir <- paste0("/data/Projects/ACC_NCDR/NCDR/BJMDATA/ACTION/NCHData/multiple_imputed/fold_", fold)
        cat(paste('Loading data for ', mode,' Model Fold ', fold, '\n', sep=''))
        
        glm_predsB.df <- read.csv(file = paste0(data_dir, "/glm_preds_trainB_", mode, ".csv"), header = TRUE)
        xgb_predsB.df <- read.csv(file = paste0(data_dir, "/xgb_preds_trainB_", mode, ".csv"), header = TRUE)
        nn_predsB.df <- read.csv(file =  paste0(data_dir,  "/nn_preds_trainB_", mode, ".csv"), header = TRUE)
        
        glm_predsC.df <- read.csv(file = paste0(data_dir, "/glm_preds_trainC_", mode, ".csv"), header = TRUE)
        xgb_predsC.df <- read.csv(file = paste0(data_dir, "/xgb_preds_trainC_", mode, ".csv"), header = TRUE)
         nn_predsC.df <- read.csv(file =  paste0(data_dir, "/nn_preds_trainC_", mode, ".csv"), header = TRUE)
        
        train.data <- data.frame(glm = glm_predsB.df$glm, xgb = xgb_predsB.df$xgb, nn = nn_predsB.df$NN)
        train.labels <- glm_predsB.df$DCStatus
        
        test.data <- data.frame(glm = glm_predsC.df$glm, xgb = xgb_predsC.df$xgb, nn = nn_predsC.df$NN)
        test.labels <- glm_predsC.df$DCStatus
        
        cat(paste('XGB ', mode,' Meta ROC, Fold ', fold, '\n', sep=''))
        #xgb meta
        xgb.model <- xgboost(data.matrix(train.data), as.numeric(train.labels), verbose=0, nrounds=250, 
                             eta=0.1, max.depth=2, nthread=64, objective='binary:logistic',
                             save_period = NULL, save_name = NULL)
        xgb.resp <- predict(xgb.model, data.matrix(test.data))
        xgb.roc <- roc.curve(scores.class0 = xgb.resp, weights.class0 = test.labels, curve = T)
        print(xgb.roc)
        xgb.vars <- xgb.importance(feature_names=colnames(test.data), model=xgb.model)
        xgb.f <- allROC_par(as.numeric(xgb.resp), as.numeric(test.labels))[[1]]
        
        preds.df <- test.data
        preds.df$DCStatus <- test.labels
        preds.df$IDX <- glm_predsC.df$IDX
        preds.df$meta <- xgb.resp
        save(preds.df, file = paste0(data_dir, '/trainCpreds_fig_', mode, '.Rdata'))
        
        print(xgb.vars)
        
        write.csv(xgb.vars,
                  file = paste0(data_dir, '/importance_meta_', mode, '.csv'),
                  row.names = FALSE)

    }
}

Loading data for mcnamara Model Fold 1
XGB mcnamara Meta ROC, Fold 1

  ROC curve

    Area under curve:
     0.8890118 

    Curve for scores from  0.0002747152  to  0.9732066 
    ( can be plotted with plot(x) )



Warning message:
“executing %dopar% sequentially: no parallel backend registered”

   Feature       Gain     Cover Frequency
1:     xgb 0.64522906 0.4863008 0.4142661
2:      nn 0.33548335 0.3008832 0.3182442
3:     glm 0.01928759 0.2128160 0.2674897
Loading data for expanded Model Fold 1
XGB expanded Meta ROC, Fold 1

  ROC curve

    Area under curve:
     0.9008187 

    Curve for scores from  0.0001691389  to  0.998134 
    ( can be plotted with plot(x) )

   Feature       Gain     Cover Frequency
1:     xgb 0.61874392 0.4450335 0.3561828
2:      nn 0.36277003 0.3386691 0.3185484
3:     glm 0.01848605 0.2162974 0.3252688
Loading data for mcnamara Model Fold 2
XGB mcnamara Meta ROC, Fold 2

  ROC curve

    Area under curve:
     0.8865455 

    Curve for scores from  0.0001121783  to  0.9527357 
    ( can be plotted with plot(x) )

   Feature        Gain     Cover Frequency
1:     xgb 0.501831772 0.3903399 0.3417015
2:      nn 0.494165647 0.4580408 0.4351464
3:     glm 0.004002581 0.1516194 0.2231520
Loading data for expanded Model Fold 2
XGB expanded Meta ROC, F